In [1]:
import itertools
import pickle
import pandas as pd
import numpy as np

In [2]:
pd.set_option('display.max_rows', 10)
pd.set_option('display.max_columns', 100)
path = 'C:\\Users\\Documentos\\Documents\\Estudos\\C3'

In [3]:
Trips = pd.read_excel(path + 'Ops_Galactic_Shipping_Company.xlsx', sheet_name = 'Trips')
Fleets = pd.read_excel(path + 'Ops_Galactic_Shipping_Company.xlsx', sheet_name = 'Fleet')
Cargos = pd.read_excel(path + 'Ops_Galactic_Shipping_Company.xlsx', sheet_name = 'Cargo')
Passengers = pd.read_excel(path + 'Ops_Galactic_Shipping_Company.xlsx', sheet_name = 'Passengers')
Routes = pd.read_excel(path + 'Ops_Galactic_Shipping_Company.xlsx', sheet_name = 'Routes')

In [4]:
Fleets['unit_passengers_cost'] = Fleets['max_passenger_cost'] / Fleets['passengers_capacity']
Fleets['unit_cargo_cost'] = Fleets['max_cargo_cost'] / Fleets['cargo_capacity (kg)']
Fleets['unit_cargo_cost'] = Fleets['unit_cargo_cost'].fillna(0)
Fleets.at[13, 'name'] = 'Republic Cruiser1'; Fleets.at[16, 'name'] = 'Republic Cruiser2'

In [6]:
def OptimalFleet(passengers, cargos, origin_planet, destiny_planet, fleets_data, cargos_data, passengers_data, routes_data, full_criteria = False, return_all = True):
    fleets_data = fleets_data.copy()

    if type(passengers) == str:
        passengers = passengers[1:].split('_')
        n_passengers = len(passengers)
    else:
        n_passengers = 0

    if type(cargos) == str:
        #Peso dos Passengers + Cargas
        cargos_weight = pd.DataFrame({'name': cargos[1:].split('_')}).merge(cargos_data[['name', 'weight (kg)']], on = 'name', how = 'left')['weight (kg)'].sum()
        # passengers_data['weight'][passengers_data['name'].isin(passengers)].sum()   ~  A princípio iria utilizar também o peso dos passageiros, mas como a coluna de capacidade foi especificada como 'CARGO_capacity (kg)', optei por desconsiderar os passageiros.
    else:
        cargos_weight = 0

    #Distância da rota
    trip_parsec = routes_data['distance (Parsec)'][routes_data['id'] == origin_planet + '_' + destiny_planet]

    #Custo Fuel / trip_parsec
    fleets_data['10³L/trip_parsec'] = fleets_data['autonomy (Parsec/10³L)'].apply(lambda x: trip_parsec / x)  #Litros necessários para a Trip em questão
    fleets_data['fuel$/trip_parsec'] = fleets_data['10³L/trip_parsec'] * fleets_data['fuel_price ($/ 10³L)']

    #Custo
    fleets_data['trip_cargo_cost'] = fleets_data['unit_cargo_cost'] * cargos_weight                   #Custo do peso dos cargos proporcional ao max_cargo_cost
    fleets_data['trip_passengers_cost'] = fleets_data['unit_passengers_cost'] * n_passengers      #Custo do número de passengers proporcional ao max_passenger_cost
    fleets_data['total_cost'] = fleets_data[['fuel$/trip_parsec', 'trip_cargo_cost', 'trip_passengers_cost']].apply(sum, axis = 1)

    #Filters
    fleets_data = fleets_data[fleets_data['passengers_capacity'].ge(n_passengers)] #Capacidade de Passengers
    fleets_data = fleets_data[fleets_data['cargo_capacity (kg)'].ge(cargos_weight)]   #Capacidade de Carga
    fleets_data = fleets_data[fleets_data['fuel_capacity ( 10³ L )'] >= (fleets_data['10³L/trip_parsec'] * extra_fuel)] #Capacidade de Combustível ~ 5% Combustível Extra

    if len(fleets_data) == 0:
        return 'No Optimal Spaceships'

    #Order Custo / TempoManutenção / Length~Speed?
    fleets_data.sort_values(by = ['total_cost'], inplace = True)    # 1) -Custo = +Lucro
                                                                    # 2) -Manutenção = +Lucro
                                                                    # 3) -Length ~ +Velocidade | MGLT com muitos missings, então optei por utilizar o Length (Correlação MGLTxLength = -.5)
                                                                    # 4) HD_rating?
    if full_criteria:
        fleets_data.sort_values(by=['total_cost', 'MTTR', 'length'], inplace = True)
    else:
        fleets_data = fleets_data[fleets_data['total_cost'] == fleets_data['total_cost'].iloc[0]].sample(frac = 1)   #Randomizando para distribuir melhor as viagens nas diferentes naves com o mesmo custo

    if not return_all:
        fleets_data = fleets_data.iloc[0, :]
        return fleets_data['name']
    return list(fleets_data['name'])


In [ ]:
Trips['optimal_spaceship'] = Trips[['passangers', 'cargos', 'origin_planet', 'destiny_planet']].apply(
    lambda x:  OptimalFleet(x[0], x[1], x[2], x[3], Fleets, Cargos, Passengers, Routes, return_all = False), axis = 1)

Trips['Droid_isOptimal'] = Trips[['passangers', 'cargos', 'origin_planet', 'destiny_planet', 'spaceship']].apply(
    lambda x: x[4] in OptimalFleet(x[0], x[1], x[2], x[3], Fleets, Cargos, Passengers, Routes), axis = 1)

In [ ]:
Fleets['autonomy (Parsec/fuel_capacity)'] = Fleets['autonomy (Parsec/10³L)'] * Fleets['fuel_capacity ( 10³ L )']

In [ ]:
Trips = Trips.merge(Fleets[['name', 'passengers_capacity', 'autonomy (Parsec/fuel_capacity)', 'cargo_capacity (kg)', 'unit_passengers_cost',
                            'unit_cargo_cost', 'fuel_price ($/ 10³L)', 'autonomy (Parsec/10³L)']], left_on = 'spaceship', right_on = 'name', how = 'left')
Trips['cargos'] = Trips['cargos'].fillna('_')
Trips['cargos_weight'] = Trips['cargos'].apply(lambda x: pd.DataFrame({'name':x[1:].split('_')}).merge(Cargos[['name', 'weight (kg)']], on = 'name', how = 'left')['weight (kg)'].sum())
Trips = Trips.merge(Routes[['origin', 'destination', 'distance (Parsec)', 'revenue']], left_on = ['origin_planet', 'destiny_planet'], right_on = ['origin', 'destination'], how = 'left')

Trips['passangers'] = Trips['passangers'].fillna('')

Trips['isOkDroid_passenger'] = Trips['passangers'].str.count('_') <= Trips['passengers_capacity']
Trips['isOkDroid_cargos'] = Trips['cargo_capacity (kg)'] >= Trips['cargos_weight']
Trips['isOkDroid_fuel'] = Trips['autonomy (Parsec/fuel_capacity)'] >= Trips['distance (Parsec)']
Trips['isOkDroid'] = Trips[['isOkDroid_passenger', 'isOkDroid_cargos', 'isOkDroid_fuel']].apply(all, axis = 1)

Trips['Droid_cost'] = (Trips['passangers'].str.count('_') * Trips['unit_passengers_cost']) + \
                      (Trips['unit_cargo_cost'] * Trips['cargos_weight']) + \
                      ((Trips['distance (Parsec)'] / Trips['autonomy (Parsec/10³L)']) * Trips['fuel_price ($/ 10³L)'])

Trips.drop(['name', 'passengers_capacity', 'autonomy (Parsec/fuel_capacity)', 'cargo_capacity (kg)', 'unit_passengers_cost', 'unit_cargo_cost',
            'fuel_price ($/ 10³L)', 'autonomy (Parsec/10³L)', 'origin', 'destination'], axis = 1, inplace = True)

In [ ]:
Trips = Trips.merge(Fleets[['name', 'passengers_capacity', 'autonomy (Parsec/fuel_capacity)', 'cargo_capacity (kg)',
                            'unit_passengers_cost', 'unit_cargo_cost', 'fuel_price ($/ 10³L)', 'autonomy (Parsec/10³L)']],
                    left_on = 'optimal_spaceship', right_on = 'name', how = 'left')
Trips['isOkOptimalFleet_passenger'] = Trips['passangers'].str.count('_') <= Trips['passengers_capacity']
Trips['isOkOptimalFleet_cargos'] = Trips['cargo_capacity (kg)'] >= Trips['cargos_weight']
Trips['isOkOptimalFleet_fuel'] = Trips['autonomy (Parsec/fuel_capacity)'] >= Trips['distance (Parsec)']
Trips['isOkOptimalFleet'] = Trips[['isOkOptimalFleet_passenger', 'isOkOptimalFleet_cargos', 'isOkOptimalFleet_fuel']].apply(all, axis = 1)

Trips['OptimalFleet_cost'] = (Trips['passangers'].str.count('_') * Trips['unit_passengers_cost']) + \
                             (Trips['unit_cargo_cost'] * Trips['cargos_weight']) + \
                             ((Trips['distance (Parsec)'] / Trips['autonomy (Parsec/10³L)']) * Trips['fuel_price ($/ 10³L)'])

Trips.drop(['name', 'passengers_capacity', 'autonomy (Parsec/fuel_capacity)', 'cargo_capacity (kg)', 'unit_passengers_cost', 'unit_cargo_cost', 'fuel_price ($/ 10³L)', 'autonomy (Parsec/10³L)'], axis = 1, inplace = True)


In [9]:
with open('C:\\Users\\Documentos\\PycharmProjects\\FhinckCase_TransportesIntergalaticos\\Saves.pickle', 'rb') as file:
    Trips, Fleets = pickle.load(file)

In [13]:
print(Trips['revenue'][Trips['isOkDroid'] == True].sum() - Trips['Droid_cost'][Trips['isOkDroid'] == True].sum())
print(Trips['revenue'][Trips['isOkOptimalFleet'] == True].sum() - Trips['OptimalFleet_cost'][Trips['isOkOptimalFleet'] == True].sum())

108248916102.0
394816286402.0


In [16]:
"""
Improvements
"""
incs_list = [[['Corvette', ['cargo_capacity (kg)', 'passengers_capacity'], [100, 2], ['add', 'set']],
             # Corvette-Aumente em 100kg a capacidade de carga e leve 2 passageiros de uma Corvette
             ['Corvette', ['cargo_capacity (kg)'], [200], ['add']],
             # Corvette-Aumente em 200kg a capacidade de carga de uma Corvette e Star Destroyer
             ['Corvette', ['cargo_capacity (kg)', 'passengers_capacity'], [500, 0], ['add', 'set']],
             # Corvette-Aumente em 500kg a capacidade de carga e leve zero passageiros de uma Corvette
             ['Corvette', ['cargo_capacity (kg)', 'passengers_capacity'], [800, 4], ['add', 'set']],
             # Corvette-Aumente em 800kg a capacidade de carga e leve 4 passageiros de uma Corvette
             ['Corvette', ['passengers_capacity'], [1], ['add']]],
             # Corvette-Aumente um espaço para passageiro em todas as classes de naves

             [['Patrol Craft', ['fuel_capacity ( 10³ L )', 'passengers_capacity'], [0.5, 5], ['add', 'set']],
             # Patrol Craft-Aumente 5000L o tanque de cobustível e diminua para 5 a qunatidade de passageiros de uma Patrol Craft
             ['Patrol Craft', ['passengers_capacity'], [8], ['set']],
             # Patrol craft-Aumente para 8 a qunatidade de passageiros de uma Patrol craft
             ['Patrol Craft', ['passengers_capacity'], [1], ['add']]],
             # Patrol craft-Aumente um espaço para passageiro em todas as classes de naves

             [['Space Battlestation', ['cargo_capacity (kg)', 'passengers_capacity'], [100, 2], ['add', 'set']],
             # Space Battlestation-Aumente em 100kg a capacidade de carga e leve 2 passageiros de uma Space Battlestation
             ['Space Battlestation', ['cargo_capacity (kg)', 'passengers_capacity'], [100, 4], ['add', 'set']],
             # Space Battlestation-Aumente em 100kg a capacidade de carga e leve 4 passageiros de uma Space Battlestation
             ['Space Battlestation', ['cargo_capacity (kg)', 'passengers_capacity'], [100, 0], ['add', 'set']],
             # Space Battlestation-Aumente em 100kg a capacidade de carga e leve zero passageiros de uma Space Battlestation
             ['Space Battlestation', ['passengers_capacity'], [1], ['add']]],
             # Space Battlestation-Aumente um espaço para passageiro em todas as classes de naves

             [['Star Destoyer', ['fuel_capacity ( 10³ L )', 'cargo_capacity (kg)'], [1, -300], ['add', 'add']],
             # Star Destoyer-Aumente 1000L o tanque de combustível e diminua em 300kg a capacidade de carga de uma Star Destoyer
             ['Star Destoyer', ['cargo_capacity (kg)'], [200], ['add']],
             # Star Destroyer-Aumente em 200kg a capacidade de carga de uma Corvette e Star Destroyer
             ['Star Destoyer', ['passengers_capacity'], [1], ['add']]],
             # Star Destroyer-Aumente um espaço para passageiro em todas as classes de naves

             [['Star dreadnought', ['fuel_capacity ( 10³ L )'], [2], ['add']],
             # Star dreadnought-Aumente 2000L o tanque de combustível de uma Satr dreadnought
             ['Star dreadnought', ['cargo_capacity (kg)', 'fuel_capacity ( 10³ L )'], [100, -1], ['set', 'add']],
             # Star dreadnought-Aumente para 100Kg a capacidade de carga e diminua em 1000L a capacidade e combustível de uma Star dreadnought
             ['Star dreadnought', ['passengers_capacity'], [1], ['add']]],
             # Star dreadnought-Aumente um espaço para passageiro em todas as classes de naves

             [['Starfighter', ['fuel_capacity ( 10³ L )', 'cargo_capacity (kg)'], [1, -300], ['add', 'add']],
             # Starfighter-Aumente 1000L o tanque de combustível e diminua em 300kg a capacidade de carga de uma Starfighter
             ['Starfighter', ['cargo_capacity (kg)', 'fuel_capacity ( 10³ L )'], [600, -2], ['add', 'add']],
             # Starfighter-Aumente em 600kg a capacidade de carga e diminua em 2000L a capacidade e combustível de uma Starfighter
             ['Starfighter', ['passengers_capacity'], [1], ['add']]]]
             # Starfighter-Aumente um espaço para passageiro em todas as classes de naves

combos = list(itertools.product(range(0, 5),  #Corvette
                                range(0, 3),  #Patrol Craft
                                range(0, 4),  #Space Battlestation
                                range(0, 3),  #Star Destoyer
                                range(0, 3),  #Star dreadnought
                                range(0, 3))) #Starfighter

In [ ]:
def calc_profit(trips_data, fleets_data, cargos_data, passengers_data, routes_data, inc = None):
    trips_data = trips_data.copy()
    fleets_data = fleets_data.copy()
    inc = inc.copy()

    if len(inc[0]):
        for inc_data in inc:
            if not isinstance(inc_data[0], list):
                inc_data[0] = [inc_data[0]]
            for ship in inc_data[0]:
                if isinstance(inc_data[1], list):
                    for i, col in enumerate(inc_data[1]):
                        if inc_data[3][i] == 'add':
                            fleets_data.loc[fleets_data['starship_class'] == ship, col] += inc_data[2][i]
                        elif inc_data[3][i] == 'set':
                            fleets_data.loc[fleets_data['starship_class'] == ship, col] = inc_data[2][i]
                        elif inc_data[3][i] == 'prod':
                            fleets_data.loc[fleets_data['starship_class'] == ship, col] *= inc_data[2][i]
                        else:
                            pass
                else:
                    i = 0
                    col = inc_data[1]
                    if inc_data[3] == 'add':
                        fleets_data.loc[fleets_data['starship_class'] == ship, col] += inc_data[2]
                    elif inc_data[3] == 'set':
                        fleets_data.loc[fleets_data['starship_class'] == ship, col] = inc_data[2]
                    else:
                        print(f'inc_data[3] incorreto - Inc: {inc}')


    trips_data['optimal_spaceship'] = trips_data[['passangers', 'cargos', 'origin_planet', 'destiny_planet']].apply(
        lambda x: OptimalFleet(x[0], x[1], x[2], x[3], fleets_data, cargos_data, passengers_data, routes_data, return_all = False), axis = 1)

    trips_data = trips_data.merge(
        fleets_data[['name', 'passengers_capacity', 'autonomy (Parsec/fuel_capacity)', 'cargo_capacity (kg)', 'unit_passengers_cost',
                     'unit_cargo_cost', 'fuel_price ($/ 10³L)', 'autonomy (Parsec/10³L)']], left_on = 'optimal_spaceship', right_on = 'name', how = 'left')

    trips_data['isOkNewTrips'] = pd.concat(
        ((trips_data['passangers'].str.count('_') <= trips_data['passengers_capacity']),
         (trips_data['cargo_capacity (kg)'] >= trips_data['cargos_weight']),
         (trips_data['autonomy (Parsec/fuel_capacity)'] >= trips_data['distance (Parsec)'])), axis = 1).apply(all, axis = 1)

    trips_data['total_cost'] = (trips_data['passangers'].str.count('_') * trips_data['unit_passengers_cost']) + \
                               (trips_data['unit_cargo_cost'] * trips_data['cargos_weight']) + \
                               ((trips_data['distance (Parsec)'] / trips_data['autonomy (Parsec/10³L)']) * trips_data[
                                   'fuel_price ($/ 10³L)'])

    revenue = trips_data['revenue'][trips_data['isOkNewTrips'] == True].sum() - trips_data['total_cost'][trips_data['isOkNewTrips'] == True].sum()
    return (trips_data['isOkNewTrips'].mean(), revenue)


In [ ]:
inc_index = []
inc_isOkNewTrips = []
inc_profit = []
trips_data = Trips.copy()
trips_data = trips_data.loc[trips_data['year'].isin(['22ABY', '23ABY'])]
trips_data = trips_data.sample(frac = 0.005)
n = len(combos)

for it, index in enumerate(combos):
    inc = [incs_list[0][index[0]], incs_list[1][index[1]], incs_list[2][index[2]],
           incs_list[3][index[3]], incs_list[4][index[4]], incs_list[5][index[5]]]
    inc_index.append(index)
    val = calc_profit(trips_data, Fleets, Cargos, Passengers, Routes, inc = inc)
    inc_isOkNewTrips.append(val[0])
    inc_profit.append(val[1])

    if it in range(0, n, 100):
        print(f'Iteração: {it} / {n}')
        with open('Optinal1_Result.pickle', 'wb') as file:
            pickle.dump((it, inc_index, inc_isOkNewTrips, inc_profit, trips_data), file)

print(f'Best index: {inc_index[np.argmax(inc_profit)]}')
print(f'Revenue(BestIndex): {max(inc_profit)}')
print(f'Expansão revenue: {max(inc_profit) * (1 / 0.005)}')

In [ ]:

"""
Chips
"""
allships = Fleets['starship_class'].unique().tolist()
chips_list = [[[allships, ['autonomy (Parsec/fuel_capacity)', 'unit_passengers_cost', 'unit_cargo_cost'], [0.1, 1.05, 1.05], ['add', 'prod', 'prod']]],
               # Aumenta a autonomia em 0,1 e aumenta todos os custos em 5% (não afeta o preço do combustível)
               [[allships, ['unit_cargo_cost'], [1.15], ['prod']]],
               # Aumenta o custo da carga em 15%
               [[allships, ['unit_passengers_cost', 'unit_cargo_cost'], [0.95, 1.2], ['prod', 'prod']]],
               # Aumenta o custo da carga em 20% e diminui o custo do passageiro em 5%
               [[allships, ['autonomy (Parsec/fuel_capacity)', 'unit_passengers_cost', 'unit_cargo_cost'], [-0.05, 0.95, 0.95], ['add', 'prod', 'prod']]],
               # Diminui a autonomia em 0.05 e diminuis os custos em 5% ( não afeta o preco do combustível)
               [[allships, ['unit_passengers_cost', 'unit_cargo_cost'], [1.02, 1.03], ['prod', 'prod']]],
               # Aumenta o custo da  carga em 3% e aumenta o custo do passageiro em 2%
               [[allships, ['unit_passengers_cost'], [0.93], ['prod']]],
               # Diminui o custo do passageiro em 7%
               [[allships, ['unit_passengers_cost', 'unit_cargo_cost'], [0.9, 1.15], ['prod', 'prod']]],
               # Aumenta em 15% o custo da carga e dinumiu 10% o custo do passageiro
               [[allships, ['autonomy (Parsec/fuel_capacity)', 'unit_cargo_cost'], [-0.01, 1.03], ['add', 'prod']]],
               # Diminui autonomia em 0,01 e aumenta o custo de carga em 3%
               [[allships, ['unit_passengers_cost', 'unit_cargo_cost'], [1.003, 0.997], ['prod', 'prod']]],
               # Aumenta o custo do passageiro em 0.3% e diminui a o custo da carga em 0,3
               [[allships, ['unit_passengers_cost', 'unit_cargo_cost'], [0.95, 1.09], ['prod', 'prod']]],
               # Aumenta em 9% o custo da carga e diminui em 5% o custo do passageiro.
               [[allships, ['autonomy (Parsec/fuel_capacity)'], [0.1], ['add']]],
               # Aumenta a autonomia em 0,1
               [[allships, ['unit_cargo_cost'], [0.95], ['prod']]],
               # Diminui em 5% o custo da carga
               [[allships, ['autonomy (Parsec/fuel_capacity)'], [-0.15], ['add']]],
               # Diminui a autonomia em 0,15
               [[allships, ['autonomy (Parsec/fuel_capacity)', 'unit_cargo_cost'], [-0.15, 0.8], ['add', 'prod']]],
               # Diminui em 20% o custo da carga e diminui a autonomia em 0,15.
               [[]],
               # Altera a cor da espaçonave
               [[allships, ['autonomy (Parsec/fuel_capacity)', 'unit_cargo_cost'], [0.2, 0.97], ['add', 'prod']]]]
               # Aumenta a autonomia em 0,2 e diminui o custo da carga em 3%

In [ ]:

chip_index = []
chip_isOkNewTrips = []
chip_profit = []

for it, it_data in enumerate(chips_list):
    chip_index.append(it)
    val = calc_profit(Trips, Fleets, Cargos, Passengers, Routes, inc = it_data)
    chip_isOkNewTrips.append(val[0])
    chip_profit.append(val[1])

    print(f'Iteração: {it} / 15')
    with open('Optinal2_Result.pickle', 'wb') as file:
        pickle.dump((chip_index, chip_isOkNewTrips, chip_profit), file)

print(f'Best chip: {chips_list[np.argmax(chip_profit)]}')
print(f'Revenue(BestIndex): {max(chip_profit)}')